In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

def model_build():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(7, activation='softmax')
    ])

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers.Adam(learning_rate=0.0001),
        metrics=['accuracy']
    )

    return model

import cv2
import numpy as np
import tensorflow as tf

def live_face_prediction(model):
    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Webcam could not be opened.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break
        
        # Convert frame to grayscale (required for face detection)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        
        for (x, y, w, h) in faces:
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Extract the region of interest (ROI) corresponding to the face
            face_roi = gray[y:y + h, x:x + w]
            
            # Resize the ROI to match the model's expected input shape (48x48)
            face_resized = cv2.resize(face_roi, (48, 48))
            
            # Normalize pixel values to [0, 1]
            face_normalized = face_resized.astype("float32") / 255.0
            
            # Expand dimensions to add batch and channel axes (shape: 1x48x48x1)
            input_img = np.expand_dims(face_normalized, axis=0)
            input_img = np.expand_dims(input_img, axis=-1)
            
            # Make a prediction with the model
            prediction = model.predict(input_img)
            label = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

            pred_class = np.argmax(prediction, axis=1)[0]
            pred_label = label[pred_class]
            
            # Display the predicted class label near the face rectangle
            cv2.putText(frame, f"Prediction: {pred_label}", (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Show the frame with predictions
        cv2.imshow("Live Face Prediction", frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()



In [6]:
# Example usage:
# Load the weights into your model (ensure model architecture matches training)
model = model_build()
model.load_weights('model_fer2013_weights.weights.h5')
live_face_prediction(model)


1/1 [==============================] - 0s 20ms/step
